# Detecting ships from sattelite images

## Instructions
Some files are needed to run this. The data sets need to be setup with .jpg and .tfw files in the dataset subdirectories

For AIS data a use token is needed and needs to be saved as _aistoken.txt_ (Or edit the code to read another file)

## Dependencies
Load dependencies and prereqs. This list contains some dependencies that are used for training, visualization and other things and not all are strictly needed.

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from skimage import io
import warnings
warnings.filterwarnings("ignore")
from keras.models import Model, load_model

from keras.layers import Input
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import backend as K
from keras.layers import Layer

from skimage.morphology import label
from skimage.io import imread
from numpy import expand_dims

from skimage.io import imread
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
from skimage.segmentation import mark_boundaries
from skimage.util import montage
from skimage.morphology import binary_opening, disk, label
import gc; gc.enable() # memory is tight

import time
import sys

import geopandas as gpd
import requests
import json
import random
import math
import cv2
import shutil
import dateutil.parser
from datetime import timezone

Using TensorFlow backend.


## Mask-RCNN setup
This only needs to be run once, and is probably better in terminal, but the command to download the MaskRCNN git and install it are as follows

!git clone https://www.github.com/matterport/Mask_RCNN.git

os.chdir('Mask_RCNN')

#!python setup.py -q install

In [2]:
#!pip list

In [3]:
from mrcnn.visualize import display_instances
from mrcnn.visualize import display_top_masks
from mrcnn.utils import extract_bboxes
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
import mrcnn.model as modellib

## Configuration


In [5]:
model_dir = "./working/"
debug = False
model_path = model_dir + "mask_rcnn_airbus_0022.h5"

class SatteliteDataSet:
    geojson = []
    photo_timestamp = "2022-03-13T03:45:54.5Z"
    ais_timestamp = "2022-03-13T03:40:54.5Z"
    input_dir = ""
    output_dir = ""
    images = []
    ais_positions = []
    meters_per_pixel = 1.5

    def __init__(self, g, mpp, pt, aist, inp, outp):
        self.geojson = g
        self.photo_timestamp = pt
        self.ais_timestamp=aist
        self.input_dir = inp
        self.output_dir = outp
        self.meters_per_pixel = mpp
        self.images = [f for f in os.listdir(inp) if f.__contains__(".jpg")]
        
geojson_geom_october= {
	"type": "Polygon",
	"coordinates": [
	  [
		[
		  103.776588959,
		  1.28728783565
		],
		[
		  103.883718219,
		  1.28305520969
		],
		[
		  103.883668582,
		  1.2129693948
		],
		[
		  103.776639383,
		  1.21714463643
		]
	  ]
	]
}
geojson_geom_march= {
	"type": "Polygon",
	"coordinates": [
	  [
		[
		  103.777154683,
		  1.28134031089
		],
		[
		  103.87771521,
		  1.28713340842
		],
		[
		  103.877940568,
		  1.2432802019
		],
		[
		  103.777266226,
		  1.23784347532
		]
	  ]
	]
}

# Roterda DIM_PNEO3_202208311038125_MS-FS_SEN_PWOI_000054299_1_1_F_1.XML
geojson_geom_rotterdam_20220831= {
	"type": "Polygon",
	"coordinates": [
	  [
		[
		  3.82822710076,
		  51.9913692281
		],
		[
		  4.08535131497,
		  51.9962064748
		],
		[
		  4.08483412032,
		  51.9211741935
		],
		[
		  3.82866011875,
		  51.9157135572
		]
	  ]
	]
}

geojson_geom_suez_211205= {
	"type": "Polygon",
	"coordinates": [
	  [
		[
		  32.2997499145,
		  30.4288324922
		],
		[
		  32.4518439073,
		  30.4255158748
		],
		[
		  32.4525631129,
		  30.2336232151
		],
		[
		  32.2999536129,
		  30.2382962227
		]
	  ]
	]
}

geojson_geom_helsingborg_20220823= {
	"type": "Polygon",
	"coordinates": [
	  [
		[
		  12.3712024601,
		  56.2082559425
		],
		[
		  12.6036440851,
		  56.2064395847
		],
		[
		  12.6035627153,
		  56.0731710866
		],
		[
		  12.3714897273,
		  56.075519788
		]
	  ]
	]
}
      
# Actual configuration of satelite images and their area        
marchData = SatteliteDataSet(geojson_geom_march, 0.3, "2022-03-13T03:45:54.5Z", "2022-03-13T03:40:54.5Z", "./input/march_1024/", "./output/march_1024/")
octoberData = SatteliteDataSet(geojson_geom_october, 1.5, "2021-10-15T03:31:04.343049Z", "2021-10-15T03:21:04.343049Z", "./input/october_1024/", "./output/october_1024/")
suez1205Data = SatteliteDataSet(geojson_geom_suez_211205, 0.3, "2021-12-06T08:44:18.245640Z", "2021-11-01T08:34:18.245640Z", "./input/Suez_05_Dec_2021/", "./output/Suez_05_Dec_2021/")
rotterdam220831Data = SatteliteDataSet(geojson_geom_rotterdam_20220831, 1.5, "2022-08-31T10:38:12.017979Z", "2022-08-31T10:28:12.017979Z", "./input/Rotterdam_31_aug_22/", "./output/Rotterdam_31_aug_22/")
helsingborg220823Data = SatteliteDataSet(geojson_geom_helsingborg_20220823, 1.5, "2022-08-23T10:33:10.117979Z", "2022-08-23T10:23:10.117979Z", "./input/Helsinborg_23_aug_22_2/", "./output_Helsingborg_2/")

# select dataset to use
dataSet =marchData



In [6]:
class DetectorConfig(Config):    
    # Give the configuration a recognizable name  
    NAME = 'airbus'
    
    GPU_COUNT = 1
    IMAGES_PER_GPU = 9
    
    BACKBONE = 'resnet50'
    
    NUM_CLASSES = 2  # background and ship classes
    
    IMAGE_MIN_DIM = 384
    IMAGE_MAX_DIM = 384
    RPN_ANCHOR_SCALES = (8, 16, 32, 64,1)
    TRAIN_ROIS_PER_IMAGE = 64
    MAX_GT_INSTANCES = 14
    DETECTION_MAX_INSTANCES = 10
    DETECTION_MIN_CONFIDENCE = 0.95
    DETECTION_NMS_THRESHOLD = 0.0

    STEPS_PER_EPOCH = 15 if debug else 150
    VALIDATION_STEPS = 10 if debug else 125
    
    ## balance out losses
    LOSS_WEIGHTS = {
        "rpn_class_loss": 30.0,
        "rpn_bbox_loss": 0.8,
        "mrcnn_class_loss": 6.0,
        "mrcnn_bbox_loss": 1.0,
        "mrcnn_mask_loss": 1.2
    }

config = DetectorConfig()
config.display()


Configurations:
BACKBONE                       resnet50
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     9
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        10
DETECTION_MIN_CONFIDENCE       0.95
DETECTION_NMS_THRESHOLD        0.0
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 9
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  384
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  384
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [384 384   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 30.0, 'rpn_bbox_loss': 0.8, 'mrcnn_class_loss': 6.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.2}
MASK_POOL_SIZE                 14
MASK_SHAPE             

In [ ]:
class InferenceConfig(DetectorConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode='inference', 
                          config=inference_config,
                          model_dir=model_dir)

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

## AIS data


In [ ]:
def ReadAIS(token, dataset):
    headers = {
    "Authorization": f"Bearer {token}",
    'content-type' : 'application/json',
    'Prefer': 'params=single-object'
    }
    data = json.dumps({
    "geojson_geometry":dataset.geojson,
    "start_timestamp": dataset.ais_timestamp,
    "end_timestamp": dataset.photo_timestamp
    })
    position_reports = requests.post("https://db.mo.ri.se/rpc/get_position_reports",headers=headers,data=data).json()
    dfItem = pd.DataFrame.from_records(position_reports)
    #dfItem = dfItem.groupby("mmsi").last()

    df = dfItem.sort_values(['mmsi', 't'], ascending=False)
    # this will return unique by column 'type' rows indexes
    idx = df['mmsi'].drop_duplicates().index
    #this will return filtered df
    dfItem = df.loc[idx,:]
    print("Found {} unique ships from {} reports".format(len(dfItem), len(position_reports)))
    return dfItem

aistoken = open('aistoken.txt', 'r').readline()

In [ ]:
%%time
dataSet.ais_positions = ReadAIS(aistoken, dataSet)

In [ ]:
dataSet.ais_positions.head()

## Visualisation 

In [ ]:
def visualize_test(id, conf): 
    image_id = id #random.choice(test_names)
    
    # original image
#     print(image_id)
    image = imread(os.path.join(conf.input_dir, image_id))
    
    # assume square image 
    resize_factor = 1 ## ORIG_SIZE / config.IMAGE_SHAPE[0]
    
    # If grayscale. Convert to RGB for consistency.
    if len(image.shape) != 3 or image.shape[2] != 3:
        image = np.stack((image,) * 3, -1) 
#     image, window, scale, padding, crop = utils.resize_image(
#         image,
#         min_dim=config.IMAGE_MIN_DIM,
#         min_scale=config.IMAGE_MIN_SCALE,
#         max_dim=config.IMAGE_MAX_DIM,
#         mode=config.IMAGE_RESIZE_MODE)

    results = model.detect([image])
    r = results[0]
    for bbox in r['rois']: 
#         print(bbox)
        x1 = int(bbox[1] * resize_factor)
        y1 = int(bbox[0] * resize_factor)
        x2 = int(bbox[3] * resize_factor)
        y2 = int(bbox[2]  * resize_factor)
        cv2.rectangle(image, (x1,y1), (x2,y2), (77, 255, 9), 3, 1)
        width = x2 - x1 
        height = y2 - y1 
#         print("x {} y {} h {} w {}".format(x1, y1, width, height))
    fig, (ax1) = plt.subplots(1, 1, figsize=(10, 10))
    ax1.set_title(f"{image_id}")
    ax1.imshow(image)

In [ ]:
def visualize_test_ais(id, dataset): 

    image_id = id #random.choice(test_names)
    aismargin = 25

    imagefilename = os.path.join(dataset.input_dir, image_id)
    image = imread(imagefilename)
    # assume square image 
    resize_factor = 1 ## ORIG_SIZE / config.IMAGE_SHAPE[0]
    
    # If grayscale. Convert to RGB for consistency.
    if len(image.shape) != 3 or image.shape[2] != 3:
        image = np.stack((image,) * 3, -1) 

    results = model.detect([image])
    r = results[0]
    
    tfwname = imagefilename[:-3] + "tif.tfw"
    if os.path.exists(tfwname):
        with open(tfwname, "r") as file1:
            f_list = [float(line) for line in file1 if len(line.strip()) > 0]
    width = image.shape[1]
    height = image.shape[0]
    minx = f_list[4]
    maxx = f_list[4] + width*f_list[0]
    maxy = f_list[5]
    miny = f_list[5] + height*f_list[3]
    #print("xmin {} ymin {} xmax {} ymax {}".format(minx, miny, maxx, maxy))
    # font
    font = cv2.FONT_HERSHEY_SIMPLEX

    # org
    org = (50, 50)

    # fontScale
    fontScale = 0.5
    color = (200, 200, 0)
    thickness = 1
    color2 = (0, 255, 100)
    thickness2 = 2
    
    sateliteTime=dateutil.parser.isoparse(dataset.photo_timestamp)
    boats = []
    boatorgs = []
    knots2metersPerSecond = 0.5144444
    knotsToPixels = knots2metersPerSecond/dataset.meters_per_pixel
    
    for index, boat in dataset.ais_positions.iterrows():
        if boat.sog > 1:
            # boat is moving
            boatTime = dateutil.parser.isoparse(boat.t)
            boatTime = boatTime.replace(tzinfo=timezone.utc)          
            distanceMoved = boat.sog * 0.1 * knotsToPixels * (sateliteTime-boatTime).seconds
        else:
            distanceMoved = 0
        
        if boat.longitude > minx and boat.longitude < maxx and boat.latitude > miny and boat.latitude < maxy:
            boats.append(boat) # add to list for ROI handling
            #Syntax: cv2.putText(image, text, org, font, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]])
            #image = cv2.putText(image, 'OpenCV', org, font, fontScale, color, thickness, cv2.LINE_AA)
            org = (int((boat.longitude-minx)/f_list[0]),int((boat.latitude-maxy)/(f_list[3])))
            org = (int(org[0]+distanceMoved * math.sin(math.radians(boat.cog/10.0))), int(org[1]-distanceMoved * math.cos(math.radians(boat.cog/10.0))))
            boatorgs.append(org)
            image = cv2.putText(image, "{} spd {}".format(boat.mmsi, boat.sog), org, font, fontScale, color, thickness, cv2.LINE_AA)
            # represents the top left corner of image

            # End coordinate
            end_point = (int(org[0]+100*math.sin(math.radians(boat.heading))), int(org[1]+-100*math.cos(math.radians(boat.heading))))
            image = cv2.arrowedLine(image, org, end_point, color2, thickness2)
            #print("Boat found index {} long {} lat {} x {} y {}".format(index, boat.longitude, boat.latitude, org[0], org[1]))
    


    for bbox in r['rois']: 
#         print(bbox)
        x1 = int(bbox[1] * resize_factor)
        y1 = int(bbox[0] * resize_factor)
        x2 = int(bbox[3] * resize_factor)
        y2 = int(bbox[2]  * resize_factor)
        aisboat = False
        for boatorg in boatorgs:
            print("boatorg: x {} y {}".format(boatorg[0], boatorg[1]))
            if boatorg[0] < x2+aismargin and boatorg[0] > x1-aismargin and boatorg[1] > y1-aismargin and boatorg[1] < y2+aismargin:
                aisboat = True
                break
        if aisboat:
            col = (77, 255, 9)
        else:
            col = (255, 77, 9)
        cv2.rectangle(image, (x1,y1), (x2,y2), col, 2, 1)
        width = x2 - x1 
        height = y2 - y1 
        print("x {} y {} w {} h {}".format(x1, y1, width, height))

    # summary
    print(f"Image:{imagefilename}\tAISBoats:{len(boats)}\t")
    fig, (ax1) = plt.subplots(1, 1, figsize=(10, 10))
    ax1.set_title(f"{image_id}")
    ax1.imshow(image)
    return image, results

In [ ]:

visualize_test(dataSet.images[0], dataSet)

In [ ]:
im, r = visualize_test_ais(dataSet.images[14], dataSet)

In [ ]:
plt.imshow(r[0]['masks'][0])

In [ ]:
r[0]

In [ ]:
# To process an antire directory, use the images property and iterate over it and save the resulting images
for im in dataSet.images:
    image = visualize_test_ais(im, dataSet)
    isWritten = cv2.imwrite(dataSet.output_dir+im,  cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
    shutil.copy(dataSet.input_dir+im[:-3] + "tif.tfw", dataSet.output_dir+im[:-3]+"jpw")

In [ ]:
dataSet.ais_positions.head()